In [ ]:
#--------------------------------------------------------------------------------------------#
#          Universidade Federal de Santa Catarina - UFSC/Florianópolis
#             Programa de Pós Graduação em Engenharia Elétrica
#                           Engenharia Biomédica
#                    Código da Dissertação de Mestrado
#                     Discente: Vinícius Rodrigues Zanon
#                       Orientador: Cesar Ramos Rodrigues
#
#    Título: Estudo da aplicação das redes neurais pulsadas em sistemas personalizados de
#            detecção de crises epilépticas com base em sinais cardíacos. 
#
#
#  - Todos os direitos de código são reservados ao autor discente deste trabalho, sendo
#    passível de processos extrajudicias se não for devidamente referenciado.
#
#   Citação:
#
#   @monography{ZANON2024,
#       author       = {Vinicius R. {Zanon}}, 
#       title        = {Estudo da aplicação das redes neurais pulsadas em sistemas personalizados
#                       de detecção de crises epilépticas com base em sinais cardíacos},
#       school       = {Universidade Federal de Santa Catarina --- UFSC},
#       year         = {2024},
#       address      = {Florianópolis},
#       pages        = {55},
#       type         = {Mestrado},
#       note         = {Programa de Pós-graduação em Engenharia Elétrica - PPGEEL}
#   }
#
#--------------------------------------------------------------------------------------------#

In [ ]:
#--------------------------------------------------------------#
#                   Importação de Bibliotecas
#--------------------------------------------------------------#
import collections
import warnings
import pandas as pd
import numpy as np
from datetime import datetime 
import time
import matplotlib.pyplot as plt
import datetime
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install imblear
!{sys.executable} -m pip install tensorflow_addons

import seaborn as sns
import imblearn
import tensorflow_addons as tfa
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

#--------------------------------------------------------------#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score

#--------------------------------------------------------------#
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier, KerasRegressor
#from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.layers import Dropout
from keras.constraints import maxnorm
from tensorboard.plugins.hparams import api as hp
import tensorflow_addons as tfa
#--------------------------------------------------------------#
import nengo
import nengo_dl
import keras_spiking
import nengo_loihi


warnings.simplefilter("ignore")
tf.get_logger().addFilter(lambda rec: "Tracing is expensive" not in rec.msg)

In [ ]:
#--------------------------------------------------------------#
#                Declaração de Funções/Métodos
#--------------------------------------------------------------#

# Função para criar o modelo, requerido pelo KerasClassifier
def create_model(init_mode='glorot_uniform', activation = 'softmax',
                 loss = ['categorical_crossentropy'], dropout_rate=0.0,
                 weight_constraint=1                 
                 ):  
  
  global layer1, layer2, layer3, layer4, layerX
  
  layer1 = tf.keras.layers.Dense(128, input_dim = 3, activation = 'relu', kernel_constraint=maxnorm(weight_constraint), name="layer1")
  layer2 = tf.keras.layers.Dropout(dropout_rate, name="layer2")
  layer3 = tf.keras.layers.Dense(64, activation = 'relu', kernel_constraint=maxnorm(weight_constraint), name="layer3")
  layer4 = tf.keras.layers.Dense(3, activation = activation, name="layer4")
  layerX = tf.keras.layers.BatchNormalization(momentum=0.99)
  
  model = Sequential([layer1, layer2, layer3, layer4, layerX])

  # Antes de treinarmos o modelo, é necessário compilarmos com o otimizador e a função de perda
  model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])
  
  return model

In [ ]:
# SEPARAÇÃO DA BASE DE DADOS EM PACIENTES ESPECÍFICOS (EXECUTAR APENAS UMA VEZ)
'''
import os
import pandas as pd

# Leitura do arquivo CSV
nome_do_arquivo = 'C:/Users/zanon/Desktop/CODIGO/Codigos-Vini-Modificado/Sinais/teste/seizures_siena_post_norm_treino_completo.csv'
dados = pd.read_csv(nome_do_arquivo, sep=';')

# Diretório de destino
diretorio_destino = 'C:\\Users\\zanon\\Desktop\\CODIGO\\Codigos-Vini-Modificado\\Sinais\\teste\\pacientes'

# Inicialização de variáveis
paciente_atual = -1
dados_paciente = []

# Iteração sobre as linhas do DataFrame
for index, row in dados.iterrows():
    # Verifica se a coluna 'Intervalo observado [min]' é igual a 1 para indicar um novo paciente
    if row['Intervalo observado [min]'] == 1:
        # Incrementa o número do paciente antes de salvar os dados do paciente anterior
        paciente_atual += 1

        # Salva os dados do paciente anterior, se existirem
        if dados_paciente:
            df_paciente = pd.DataFrame(dados_paciente)
            caminho_arquivo = os.path.join(diretorio_destino, f'paciente_{paciente_atual}.csv')
            df_paciente.to_csv(caminho_arquivo, index=False, sep=';')
        
        # Reinicia a lista de dados do paciente
        dados_paciente = []

    # Adiciona os dados à lista do paciente atual
    dados_paciente.append(row)

# Salva os dados do último paciente
if dados_paciente:
    df_paciente = pd.DataFrame(dados_paciente)
    caminho_arquivo = os.path.join(diretorio_destino, f'paciente_{paciente_atual}.csv')
    df_paciente.to_csv(caminho_arquivo, index=False, sep=';')

'''

In [ ]:
#--------------------------------------------------------------#
#                      Programa Principal
#--------------------------------------------------------------#

# Importação dos dados com url

uri = "C:/Users/zanon/Desktop/CODIGO/Codigos-Vini-Modificado/Sinais/teste/pacientes/paciente_1.csv"    #dataset específico paciente X
nome_pasta1 = 'logs_with_cross_validation_P1/data-'
nome_pasta2 = 'logs_with_cross_validation_P1/hparam_tuning/'
type_search_flag = 1  

dados = pd.read_csv(uri, sep=';', low_memory=False)

print('\nIntervalos observados por classe')
print(dados["Crise"].value_counts())

SEED = 5
np.random.seed(SEED)
tf.random.set_seed(SEED)

#tf.debugging.set_log_device_placement(True)

troca = {'interictal' : 0,
         'preictal'   : 1,
         'ictal'      : 2,
         'postictal'  : 0}

dados['Crise_idx'] = dados.Crise.map(troca)

# separando os dados para features e para classe
x_dados = dados[ ["MediaRR [ms]",  # Average RR-intervals
                  "CSI_mod [a.u]", # Cardiac Vagal Index
                  "AT [a.u]"       # Hjorth Activity
                  ] ]

y_dados = dados["Crise"]
print('\nCabeçalho do x_dados')
print(x_dados.head())

print('\nCabeçalho do y_dados')
print(y_dados.head())

print('\nForma dos Dados')
print('x_dados: ', x_dados.shape)
print('y_dados: ', y_dados.shape)

x_train_val, x_test, y_train_val, y_test = train_test_split(x_dados, y_dados, test_size=0.2, random_state=42) # 80 - train : 20 - test   

print('x_train_val | y_train_val',x_train_val.shape, y_train_val.shape)
print('x_test | y_test', x_test.shape, y_test.shape)

In [ ]:
plt.figure(figsize=(18,8))
plt.subplot(4,1,1) 
plt.plot(x_dados['MediaRR [ms]'], color='DeepPink')
plt.ylabel('MediaRR [ms]')
plt.grid()
plt.subplot(4,1,2) 
plt.plot(x_dados['CSI_mod [a.u]'], color='DarkSlateGray')
plt.ylabel('CSI_mod [a.u]')
plt.grid()
plt.subplot(4,1,3) 
plt.plot(x_dados['AT [a.u]'], color='Green')
plt.ylabel('AT [a.u]')
plt.grid()
plt.subplot(4,1,4) 
plt.plot(y_dados, label='Output', color='cornflowerblue')
plt.ylabel('Períodos')
plt.xlabel('Tempo [s]')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(14,20)) 
plt.subplot(5,2,1) 
sns.distplot(dados['MediaRR [ms]'],color='DeepPink') 
plt.subplot(5,2,2) 
sns.boxplot(dados['MediaRR [ms]'],color='DeepPink') 
plt.subplot(5,2,3) 
sns.distplot(dados['CSI_mod [a.u]'],color='DarkSlateGray') 
plt.subplot(5,2,4) 
sns.boxplot(dados['CSI_mod [a.u]'],color='DarkSlateGray') 
plt.subplot(5,2,5) 
sns.distplot(dados['AT [a.u]'],color='Green') 
plt.subplot(5,2,6) 
sns.boxplot(dados['AT [a.u]'],color='Green') 
plt.tight_layout() 
plt.show()

In [ ]:
# verificando a correlaçao entre as variaveis via método de pearson

corr = x_dados.corr(method='pearson')
plt.figure(figsize=(5,5))
sns.heatmap(corr, vmin = -1, vmax=1, linewidth=0.001, square=True, annot=True, cmap='YlGnBu', linecolor='white')
plt.title('Correlação entre as métricas')
plt.show()


In [ ]:
# Avaliar a assimetria (se tiver: excluir variáveis muito assimétricas - valores muito altos ou muito baixos)
from scipy.stats import skew
skewed_feats = x_dados.apply(lambda x: skew(x.dropna()))
print(skewed_feats.sort_values(ascending=False))

In [ ]:
# Verificar features constantes (com variância zero)
constant_features = [feat for feat in x_dados.columns if x_dados[feat].std() == 0]
print(constant_features)

In [ ]:
# Padronização / Regularização dos Dados de Treinamento
sc = StandardScaler()
sc.fit(x_dados)
x_dados = pd.DataFrame(sc.transform(x_dados), columns=x_dados.columns)
print('Média x_dados: \n', x_dados.mean()) # media deve ser 0 
print('\nDesvio Padrão x_dados: \n', x_dados.var()) # desvio padrão deve ser 1

In [ ]:
x_dados_count = dados['Intervalo observado [min]']
print(type(x_dados_count))

contagem_pacientes = x_dados_count.value_counts()
print("Intervalo analisado [min]    [count]:", contagem_pacientes.head(1))
print("\nNumero de pacientes: ", contagem_pacientes.iloc[0])


In [ ]:
# Cria o Modelo Clássico de Rede
model = create_model()
model.summary()

# Avalia consumo de energia do modelo atual.
energy = keras_spiking.ModelEnergy(model)
energy.summary(print_warnings=False)

In [ ]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


In [ ]:
# Training the model

def run_train(run_dir, hparams):
    
    #Adjust data train
    x_dados_ = x_train_val
    y_dados_ = y_train_val

    #'Ictal', 'Interictal', 'Pre-Ictal' -> 0, 1 , 2
    label_encoder = preprocessing.LabelEncoder()
    label_encoder.fit(y_dados_)
    y_dados_ = label_encoder.transform(y_dados_)

    label_encoder_test = preprocessing.LabelEncoder()
    label_encoder_test.fit(y_test)
    y_test_ = label_encoder_test.transform(y_test)

    x_dados_ = x_dados_.to_numpy()
    x_test_ = x_test.to_numpy()

    x_test_ = x_test_.reshape((x_test_.shape[0], 1, -1))
    y_test_ = y_test_.reshape((y_test_.shape[0], 1, -1))

    x_dados_ = x_dados_.reshape((x_dados_.shape[0], 1, -1))
    y_dados_ = y_dados_.reshape((y_dados_.shape[0], 1, -1))

    x_dados_ = np.asarray(x_dados_).astype(np.float32)
    y_dados_ = np.asarray(y_dados_).astype(np.int64)

    x_test_ = np.asarray(x_test_).astype(np.float32)
    y_test_ = np.asarray(y_test_).astype(np.int64)

    print('RUN TRAIN:')
    print('Shape x_dados_train: ' , x_dados_.shape)
    print('Shape y_dados_train: ', y_dados_.shape)
 
    model = create_model(init_mode='glorot_uniform', activation = 'softmax', dropout_rate=hparams[HP_DROPOUT_RATE],
                 weight_constraint=hparams[HP_W_CONSTRAINT])
    
    # Convert model simple way
    aux_act = hparams[HP_ACTIVATION]
    if aux_act == 'LIF':
        act = nengo.LIF()
    elif aux_act == 'Izhikevich':
        act = nengo.Izhikevich()
    elif aux_act == 'SpikingRectifiedLinear':
        act = nengo.SpikingRectifiedLinear()
    elif aux_act == 'RectifiedLinear':
        act = nengo.RectifiedLinear()
    else :
        act = nengo.RectifiedLinear()

    
    converter = nengo_dl.Converter(model, swap_activations={tf.keras.activations.relu: act},
                                   scale_firing_rates=hparams[HP_SCALE_FIRING_RATES],
                                   synapse=hparams[HP_SYNAPSE])
    
    
    #print(converter.verify()) # if return equals 'true' it means it's the correct conversion

    # Create data dictionaries to input data (train, validation and test)
    # OBS: converter.model
    train_inputs = {converter.inputs[converter.model.layers[0].get_output_at(-1)]: x_dados_[train]}
    train_targets = {converter.outputs[converter.model.output]: y_dados_[train]}

    val_inputs = {converter.inputs[converter.model.layers[0].get_output_at(-1)]: x_dados_[validation]}
    val_targets = {converter.outputs[converter.model.output]: y_dados_[validation]}

    test_inputs = {converter.inputs[converter.model.layers[0].get_output_at(-1)]: x_test_}
    test_targets = {converter.outputs[converter.model.output]: y_test_}

    # get input/output objects
    nengo_input = converter.inputs[converter.model.layers[0].get_output_at(-1)]
    nengo_output = converter.outputs[converter.model.output]
 
    # Chose the optimizer and learning rate
    learn_rate = hparams[HP_LR]
    ep = hparams[HP_EPOCHS]
    
    def get_callbacks(log_dir):
            return [
                #tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20),
                tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
                hp.KerasCallback(log_dir, hparams),
              ]
    
    # Creates names to save the parameters of trainig and logs to tensorboard
    log_dir = nome_pasta1 + run_dir
    
    # Training the model
    do_training = True
    if do_training:
        print('#--------Training---------#')
        mini_batch = hparams[HP_BATCH_SIZE]
        # set some options to speed up simulation
        with converter.net:
            nengo_dl.configure_settings(stateful=False)
       
        ## OBS:Use converter.net
        # run training nengo converter network
        with nengo_dl.Simulator(converter.net, minibatch_size=mini_batch) as sim:
            # Evaluate before training (need compile before)
            # Evaluate: Compute the loss and metric values for the network.
            sim.compile(
                loss={nengo_output: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
                metrics={nengo_output: [f1_metric]})
                #metrics={nengo_output: [tf.metrics.sparse_categorical_accuracy, f1_metric]})
            
            # Evaluate before training
            # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch
            evaluate_b = sim.evaluate(test_inputs, test_targets, verbose=1)

            # Get start time training
            start_train_time = time.time()
            
            # Run training
            sim.compile(
                optimizer=tf.optimizers.Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-8),
                loss={nengo_output: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
                metrics={nengo_output: [f1_metric]})
            
            history = sim.fit(train_inputs, train_targets, 
                validation_data=(val_inputs, val_targets),
                epochs=ep,
                callbacks = get_callbacks(log_dir)
            )
            
            # Get end time training
            elapsed_train_time = time.time() - start_train_time
            elapsed_train_time_m = elapsed_train_time / 60

            # Save the parameters to file
            params_file = sim.save_params(log_dir)

            # Evaluate after training
            sim.compile(
                loss={nengo_output: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
                metrics={nengo_output: [f1_metric]})
            
            # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch
            evaluate_a = sim.evaluate(test_inputs, test_targets, verbose = 1)

    else:
        # download pretrained weights
        print("Loaded pretrained weights")
        log_dir = "data-keras_to_snn_params_zanon-1fold-0run-5wc-0.0dp-100bs-200ep-SpikingRectifiedLinearact-5sfr-0.001syn-0.001lr-20230906-210620"
        with nengo_dl.Simulator(converter.net, seed=0) as sim:
            sim.load_params(log_dir)   

    ## Plot metrics before and after training
    print('\n#-------- Metrics Data Before Training --------#')
    print(evaluate_b)


    print('\n#-------- Metrics Data After Training --------#')
    print(evaluate_a)


    print("\n>>> [DEBUG] History Keys: ")
    print(history.history.keys())

    
    loss = history.history['loss']
    val_loss = history.history['val_loss']
        
    f1_score = history.history['probe_f1_metric']
    val_f1_score = history.history['val_probe_f1_metric']
    epochs_range = range(len(f1_score))

    plt.figure(figsize=(14, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, f1_score, label='Training f1_score')
    plt.plot(epochs_range, val_f1_score, label='Validation f1_score')
    plt.legend(loc='upper right')
    plt.xlabel("Epochs")
    plt.ylabel("f1_score")
    plt.title('Training and Validation f1_score')
    plt.ylim(0,1)
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title('Training and Validation Loss')
    plt.ylim(0,1)
    plt.grid()
    plt.show()
    
    # Run predict complete data
    
    run_predict(converter, log_dir, x_test_, y_test_)
    return evaluate_a, elapsed_train_time_m

def run(run_dir, hparams):
  with tf.summary.create_file_writer(nome_pasta2 + run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    eval_a, train_time = run_train(run_dir, hparams)
    print("\n>>> [DEBUG]: ")
    print(eval_a)
    tf.summary.scalar(METRIC_LOSS, float(eval_a["loss"]), step=1)
    tf.summary.scalar(METRIC_F1, float(eval_a["probe_f1_metric"]), step=1)
    tf.summary.scalar(TIME, float(train_time), step=1)
    
    

In [ ]:
# Now, run (don't train) and load the parameters trained before. Change the parameters to get better results 

def run_predict(nengo_converter, log_dir, x_test_, y_test_, n_test=365229):

    #Adjust data teste
    x_dados_test = x_test_
    y_dados_test = y_test_
    
    # get input/output objects
    nengo_input = nengo_converter.inputs[nengo_converter.model.layers[0].get_output_at(-1)]
    nengo_output = nengo_converter.outputs[nengo_converter.model.output]  
    
    with nengo_converter.net:
        nengo_dl.configure_settings(stateful=False)
    
    # new simulation to do predict with whatever n_test
    with nengo_dl.Simulator(nengo_converter.net, seed=0) as sim:
        
        sim.load_params(log_dir)
        ## Predict: Generate output predictions for the input TEST samples (just with "n_test" value)
        data = sim.predict(x_dados_test[0:n_test])    
    
    # The accuracy can be compute like this function below with the specific n_teps, but if you use sim.evalute function
    # you compute the accuracy with all TEST data.
    # So, compute accuracy on test data, using output of network on last timestep
    predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
    accuracy = (predictions == y_dados_test[:n_test, 0, 0]).mean()
    print(f"Test accuracy: {100 * accuracy:.2f}%")
    
    print("\n>>> [DEBUG]: ")
    print("Tipo de Dados: ")
    dtype_array1 = y_dados_test.dtype
    dtype_array2 = predictions.dtype
    #print(f"Tipo de conteúdo do array (y_dados_test, predictions): {dtype_array1, dtype_array2}")
    
    print("#-------------------------------------------------------------------------------------------------------#")
    
    con_matrix = confusion_matrix(y_dados_test[:n_test, 0, 0], predictions)
    print("Os valores da matriz de confusão são: \n", con_matrix)

    print("#-------------------------------------------------------------------------------------------------------#")
    print("Dados estatísticos:")
    print("#-------------------------------------------------------------------------------------------------------#")
    epsilon = 1e-9  # Valor pequeno para suavização/smoothing Laplaciana (garantir não divisão por zero)

    print("Amostras totais: Normal:", con_matrix.sum(axis=1)[1], ",",
          "Pre-Ictal:", con_matrix.sum(axis=1)[2],",",
          "Ictal:", con_matrix.sum(axis=1)[0])#,",",
          #"Post-Ictal", matriz_confusao_DT.sum(axis=1)[2])

    print("#-------------------------------------------------------------------------------------------------------#")
    FP = (con_matrix.sum(axis=0) - np.diag(con_matrix))

    print("Falsos-Positivos: Normal:", FP[1], ",",
          "Pre-Ictal:", FP[2],",",
          "Ictal:", FP[0])#,",",
          #"Post-Ictal:", FP[2],'\n')

    FN = con_matrix.sum(axis=1) - np.diag(con_matrix)

    print("Falsos-Negativos: Normal:", FN[1], ",",
          "Pre-Ictal:", FN[2], ",",
          "Ictal:", FN[0])#,",",
          #"Post-Ictal:", FN[2],'\n')

    TP = np.diag(con_matrix)
    print("Verdadeiros-Positivos: Normal:", TP[1], ",",
          "Pre-Ictal:", TP[2], ",",
          "Ictal:", TP[0])#,",",
          #"Post-Ictal:", TP[2],'\n')

    TN = con_matrix.sum() - (FP + FN + TP)

    print("Verdadeiros-Negativos: Normal", TN[1],
          ",",
          "Pre-Ictal:", TN[2],
          ",",
          "Ictal:", TN[0])#,
          #",",
         #"Post-Ictal:", TN[2])

    print("#-------------------------------------------------------------------------------------------------------#")

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN+epsilon) * 100
    print("Taxa de Verdadeiros Positivos (sensibilidade): Normal: %.2f" %TPR[1],"%", ",", "Pre-Ictal: %.2f" %TPR[2],"%",
          ",","Ictal: %.2f" %TPR[0],"%")#,",","Post-Ictal: %.2f" %TPR[2],"%","\n")

    # Specificity or true negative rate
    TNR = TN/(TN+FP+epsilon) * 100
    print("Taxa de Verdadeiros Negativos (especificidade): Normal: %.2f" %TNR[1],"%", ",", "Pre-Ictal: %.2f" %TNR[2],"%",
          ",","Ictal: %.2f" %TNR[0],"%")#,",","Post-Ictal: %.2f" %TNR[2],"%")
    print("#-------------------------------------------------------------------------------------------------------#")

    # Precision or positive predictive value
    PPV = TP/(TP+FP+epsilon) * 100
    print("Valor Preditivo Positivo (precisão): Normal: %.2f" %PPV[1],"%", ",", "Pre-Ictal: %.2f" %PPV[2],"%", ",",
          "Ictal: %.2f" %PPV[0],"%")#, "Post-Ictal: %.2f" %PPV[2],"%","\n")

    # Negative predictive value
    NPV = TN/(TN+FN+epsilon) * 100
    print("Valor Preditivo Negativo: Normal: %.2f" %NPV[1],"%", ",", "Pre-Ictal: %.2f" %NPV[2],"%", ",",
          "Ictal: %.2f" %NPV[0],"%")#, "Post-Ictal: %.2f" %NPV[2],"%")

    print("#-------------------------------------------------------------------------------------------------------#")

    # Fall out or false positive rate
    FPR = FP/(FP+TN+epsilon)
    print("Taxa de Falsos Positivos: Normal: %.5f" %FPR[1], ",", "Pre-Ictal: %.5f" %FPR[2], ",",
          "Ictal: %.5f" %FPR[0])#, "Post-Ictal: %.5f" %FPR[2],"\n")

    # False negative rate
    FNR = FN/(TP+FN+epsilon)
    print("Taxa de Falsos Negativos: Normal: %.5f" %FNR[1], ",", "Pre-Ictal: %.5f" %FNR[2], ",",
          "Ictal: %.5f" %FNR[0])#, "Post-Ictal: %.5f" %FNR[2],"\n")

    # False discovery rate: expressa a proporção esperada de hipóteses nulas rejeitadas erroneamente e possibilita o controle dos falsos positivos
    FDR = FP/(TP+FP+epsilon)
    print("Taxa de descoberta falsa: Normal: %.2f" %FDR[1],"%", ",", "Pre-Ictal %.2f" %FDR[2],"%", ",",
          "Ictal: %.2f" %FDR[0])#, "Post-Ictal: %.2f" %FDR[2],"%")

    print("#-------------------------------------------------------------------------------------------------------#")
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN+epsilon) * 100
    print("Taxa de Acertos (acurácia): Normal: %.2f" %ACC[1],"%", ",", "Pre-Ictal %.2f" %ACC[2],"%", ",",
          "Ictal: %.2f" %ACC[0],"%")#, "Post-Ictal: %.2f" %ACC[2],"%")
    print("#-------------------------------------------------------------------------------------------------------#")

    print("#-------------------------------------------------------------------------------------------------------#")
    # Overall F1-Score
    F1 = 2*((PPV*TPR)/(PPV+TPR+epsilon))
    print("Taxa de Acertos (F1-Score): Normal: %.2f" %F1[1],"%", ",", "Pre-Ictal %.2f" %F1[2],"%", ",",
          "Ictal: %.2f" %F1[0],"%")#, "Post-Ictal: %.2f" %F1[2],"%")
    print("#-------------------------------------------------------------------------------------------------------#")
    

In [ ]:
# Definir quais são os parâmetros a serem variados (Randomized Search or Grid Search)
# Configurar estilo de Search => 0 - Grid Search (Discrete) | 1 - Randomized Search (Interval)

# Parâmetros constantes da rede do Sanchotene
HP_W_CONSTRAINT = hp.HParam('w_constraint', hp.Discrete([5]))
HP_DROPOUT_RATE = hp.HParam('dropout_rate', hp.Discrete([0]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([100])) 

if type_search_flag == 0:                                                             # Usado no método de grid search
    HP_EPOCHS = hp.HParam('epochs', hp.Discrete([200]))                               # 200
    HP_ACTIVATION = hp.HParam('activation', hp.Discrete(["SpikingRectifiedLinear"])) 
    HP_SCALE_FIRING_RATES = hp.HParam('firing_rates', hp.Discrete([5]))               # 5
    HP_SYNAPSE = hp.HParam('synapse', hp.Discrete([0.001]))                           #0.001
    HP_LR = hp.HParam('learning_rate', hp.Discrete([0.001]))                          # 0.001
elif type_search_flag == 1:                                                        # Usado no método de randomized search
    HP_EPOCHS = hp.HParam('epochs', hp.IntInterval(100, 200))                          # 100, 200
    HP_LR = hp.HParam('learning_rate', hp.RealInterval(0.001, 0.01))                  # 0.001, 0.01
    HP_ACTIVATION = hp.HParam('activation', hp.Discrete(["LIF", "Izhikevich", "SpikingRectifiedLinear"])) 
    HP_SCALE_FIRING_RATES = hp.HParam('firing_rates', hp.IntInterval(2, 5))           # 2, 5
    HP_SYNAPSE = hp.HParam('synapse', hp.RealInterval(0.001, 0.005))                  #0.001, 0.005


## Define as métricas de retorno -> Lembrar que é na predição..
METRIC_LOSS = 'loss'
METRIC_F1 = 'f1score'
TIME = 'time'

#[HP_W_CONSTRAINT, HP_DROPOUT_RATE, HP_BATCH_SIZE, HP_BATCH_SIZE, HP_EPOCHS, HP_ACTIVATION, HP_SCALE_FIRING_RATES, HP_SYNAPSE, HP_LR]

with tf.summary.create_file_writer(nome_pasta2).as_default():
  hp.hparams_config(
    hparams=[HP_W_CONSTRAINT, HP_DROPOUT_RATE, HP_BATCH_SIZE, HP_EPOCHS, HP_ACTIVATION, HP_SCALE_FIRING_RATES, HP_SYNAPSE, HP_LR],
    metrics=[
             hp.Metric(METRIC_LOSS, display_name='Loss'),
             hp.Metric(METRIC_F1, display_name='F1-Score'),
             hp.Metric(TIME, display_name='Time'),
            ])

# Define the K-fold Cross Validator
skfold = StratifiedKFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
aux_acc = np.zeros(0)

print("\n#----------------------- CONVERTER SNN TRAINING --------------------------#")
start_train_all = time.time()



for train, validation in skfold.split(x_train_val, y_train_val):

    print("\n#---------------------------------------------------------#")
    print("Fold: ", fold_no)
    start_train_fold = time.time()
    session_num = 0
    
    if type_search_flag == 0: # Usa método - Discrete()
        for w_constraint in HP_W_CONSTRAINT.domain.values: 
            for dropout_rate in HP_DROPOUT_RATE.domain.values: 
                for batch_size in HP_BATCH_SIZE.domain.values: 
                    for epochs in HP_EPOCHS.domain.values:
                        for activation in HP_ACTIVATION.domain.values:
                            for scale_firing_rates in HP_SCALE_FIRING_RATES.domain.values: 
                                for synapse in HP_SYNAPSE.domain.values: 
                                    for lr in HP_LR.domain.values:
                                        hparams = {
                                            HP_W_CONSTRAINT: w_constraint, 
                                            HP_DROPOUT_RATE: dropout_rate, 
                                            HP_BATCH_SIZE: batch_size,
                                            HP_EPOCHS: int("%d"%int(epochs)), # 1 casa
                                            HP_ACTIVATION: activation,
                                            HP_SCALE_FIRING_RATES: int("%d"%int(scale_firing_rates)), # 1 casa
                                            HP_SYNAPSE:float("%.3f"%float(synapse)), # 3 casas
                                            HP_LR: float("%.3f"%float(lr)), # 3 casas
                                        }
    
                                        name = "keras_to_snn_params_zanon-" + str(fold_no) + "fold-" + str(session_num) + "run-" + str(hparams[HP_W_CONSTRAINT]) + "wc-"+ str(hparams[HP_DROPOUT_RATE]) + "dp-" + str(hparams[HP_BATCH_SIZE]) + "bs-"+ str(hparams[HP_EPOCHS]) + "ep-"+ str(hparams[HP_ACTIVATION]) + "act-" + str(hparams[HP_SCALE_FIRING_RATES]) + "sfr-" + str(hparams[HP_SYNAPSE]) + "syn-" + str(hparams[HP_LR]) + "lr-"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
                                        run_name = "run-%d" % session_num 
                                        print('Starting trial: %s' % run_name)
                                        print({h.name: hparams[h] for h in hparams})
                                        run(name, hparams)
                                        session_num += 1
                                        
    elif type_search_flag == 1: # Usa método - Interval()
        for w_constraint in HP_W_CONSTRAINT.domain.values: 
            for dropout_rate in HP_DROPOUT_RATE.domain.values: 
                for batch_size in HP_BATCH_SIZE.domain.values: 
                    for epochs in tf.linspace(HP_EPOCHS.domain.min_value, HP_EPOCHS.domain.max_value, 2):
                        for activation in HP_ACTIVATION.domain.values:
                            for scale_firing_rates in tf.linspace(HP_SCALE_FIRING_RATES.domain.min_value, HP_SCALE_FIRING_RATES.domain.max_value, 2): 
                                for synapse in tf.linspace(HP_SYNAPSE.domain.min_value, HP_SYNAPSE.domain.max_value, 2): 
                                    for lr in tf.linspace(HP_LR.domain.min_value, HP_LR.domain.max_value, 2):
                                        hparams = {
                                            HP_W_CONSTRAINT: w_constraint, 
                                            HP_DROPOUT_RATE: dropout_rate, 
                                            HP_BATCH_SIZE: batch_size,
                                            HP_EPOCHS: int("%d"%int(epochs)), # 1 casa
                                            HP_ACTIVATION: activation,
                                            HP_SCALE_FIRING_RATES: int("%d"%int(scale_firing_rates)), # 1 casa
                                            HP_SYNAPSE:float("%.3f"%float(synapse)), # 3 casas
                                            HP_LR: float("%.3f"%float(lr)), # 3 casas
                                        }
    
                                        name = "keras_to_snn_params_zanon-" + str(fold_no) + "fold-" + str(session_num) + "run-" + str(hparams[HP_W_CONSTRAINT]) + "wc-"+ str(hparams[HP_DROPOUT_RATE]) + "dp-" + str(hparams[HP_BATCH_SIZE]) + "bs-"+ str(hparams[HP_EPOCHS]) + "ep-"+ str(hparams[HP_ACTIVATION]) + "act-" + str(hparams[HP_SCALE_FIRING_RATES]) + "sfr-" + str(hparams[HP_SYNAPSE]) + "syn-" + str(hparams[HP_LR]) + "lr-"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
                                        run_name = "run-%d" % session_num 
                                        print('Starting trial: %s' % run_name)
                                        print({h.name: hparams[h] for h in hparams})
                                        run(name, hparams)
                                        session_num += 1

        
    elapsed_train_fold = (time.time() - start_train_fold)/60
    print("\nTime training fold (min): ", elapsed_train_fold)
    fold_no = fold_no + 1
    
print("\n#--------------------------------#")                                
elapsed_train_all = (time.time() - start_train_all)/60
print("\nTime training all folds (min): ", elapsed_train_all)      


In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard

In [ ]:
#%tensorboard --logdir logs_with_cross_validation_P1/hparam_tuning

In [ ]:
def energy(model_energy):
    print("\n#-------------------------------------------------------------------------------------------------------#")
    print("RNA Original Model:")
    model_energy.summary()
    energy = keras_spiking.ModelEnergy(model_energy, example_data=np.ones((1500, 3)))
    print("\n#-------------------------------------------------------------------------------------------------------#")
    print("General")
    energy.summary(print_warnings=False)
    
    print("\n#-------------------------------------------------------------------------------------------------------#")
    print("Estimation energy: CPU X GPU")
    # Detalhado
    energy.summary(columns=(
        "name",
        "rate", 
        "synop_energy cpu",
        "synop_energy arm",
        "neuron_energy cpu",
        "neuron_energy arm",
    ),print_warnings=False, line_length = 100)
    #Resumido
    print("\n")
    energy.summary(columns=("name", "energy cpu", "energy gpu"), print_warnings=False)
    
    print("\n#-------------------------------------------------------------------------------------------------------#")
    print("Estimation energy: Loihi X Spinnaker")
    #Detalhado
    energy.summary(columns=(
        "name",
        "rate",
        "synop_energy loihi",
        "neuron_energy loihi",
        "synop_energy spinnaker2",
        "neuron_energy spinnaker2",
    ),
    print_warnings=False, line_length = 120)
    #Resumido
    print("\n")
    energy.summary(columns=("name", "energy loihi", "energy spinnaker2"), print_warnings=False)

    
energy(model)    

    